In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";   

In [2]:
import ktrain
from ktrain import text

using Keras version: 2.2.4-tf


## Load the Data Into Arrays

In [3]:
categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


## STEP 1: Preprocess Data

In [4]:
trn, val, preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          class_names=train_b.target_names,
                                          preprocess_mode='distilbert',
                                          maxlen=350)

preprocessing train...
language: en


preprocessing test...
language: en


## STEP 2:  Build a Model and Wrap in Learner

In [5]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained word vectors [https://arxiv.org/abs/1712.09405]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face [https://arxiv.org/abs/1910.01108]


In [6]:
model = text.text_classifier('distilbert', train_data=trn, preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [7]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

## STEP 3: Train Model

In [8]:
learner.fit_onecycle(3e-5, 4)



begin training using onecycle policy with max lr of 3e-05...
Train for 377 steps, validate for 251 steps
Epoch 1/4
377/377 [==============================] - 69s 183ms/step - loss: 0.6874 - accuracy: 0.7882 - val_loss: 0.3342 - val_accuracy: 0.8948
Epoch 2/4
377/377 [==============================] - 58s 154ms/step - loss: 0.1288 - accuracy: 0.9659 - val_loss: 0.2021 - val_accuracy: 0.9341
Epoch 3/4
377/377 [==============================] - 60s 159ms/step - loss: 0.0538 - accuracy: 0.9849 - val_loss: 0.1343 - val_accuracy: 0.9621
Epoch 4/4
377/377 [==============================] - 61s 161ms/step - loss: 0.0176 - accuracy: 0.9973 - val_loss: 0.1415 - val_accuracy: 0.9634


## Predict on New Data

In [9]:
p = ktrain.get_predictor(model, preproc)

In [11]:
p.predict("There is a problem with my computer monitor's resolution.  Everything is blurry.")

'comp.graphics'